In [15]:

import datetime
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import sys
import re
import string
import json
import os
from datasets import Dataset

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

from sklearn.model_selection import train_test_split
#from treeinterpreter import treeinterpreter as ti
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn import svm
from sklearn.svm import SVR 

from sklearn.metrics import mean_squared_error
from math import sqrt

In [22]:
import nltk
nltk.download('vader_lexicon')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata
sentiment_i_a = SentimentIntensityAnalyzer()

from nltk.corpus import subjectivity
from nltk.sentiment.util import *

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/yuchen/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/yuchen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yuchen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/yuchen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
from transformers import BertTokenizer

# Preprocessing function
def preprocess_tweet(tweet):
    tweet = re.sub(r'http\S+|www\S+', '[URL]', tweet)  # Replace URLs with [URL]
    tweet = re.sub(r'#', '', tweet)  # Remove '#' from hashtags
    tweet = re.sub(r'([!?.])\1+', r'\1', tweet)  # Normalize repeated punctuation
    return tweet

# Load dataset
splits = {'train': 'sent_train.csv', 'validation': 'sent_valid.csv'}
train_df = pd.read_csv("hf://datasets/zeroshot/twitter-financial-news-sentiment/" + splits["train"])
validation_df = pd.read_csv("hf://datasets/zeroshot/twitter-financial-news-sentiment/" + splits["validation"])

# Apply preprocessing
train_df['processed_text'] = train_df['text'].apply(preprocess_tweet)
validation_df['processed_text'] = validation_df['text'].apply(preprocess_tweet)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")

# Tokenize the preprocessed text
train_encodings = tokenizer(list(train_df['processed_text']), padding=True, truncation=True, max_length=128)
validation_encodings = tokenizer(list(validation_df['processed_text']), padding=True, truncation=True, max_length=128)
train_encodings['labels'] = train_df['label']
validation_encodings['labels'] = validation_df['label']
# Ready for training


In [16]:
train_dataset = Dataset.from_dict(train_encodings)
validation_dataset = Dataset.from_dict(validation_encodings)

In [17]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis", num_labels=3)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset
)

trainer.train()


/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.618100,0.474916
2,0.404700,0.453944


TrainOutput(global_step=1194, training_loss=0.4830047377389879, metrics={'train_runtime': 1313.7378, 'train_samples_per_second': 14.528, 'train_steps_per_second': 0.909, 'total_flos': 971008762470516.0, 'train_loss': 0.4830047377389879, 'epoch': 2.0})

In [18]:
trainer.save_model('/Users/yuchen/Downloads/') 

In [145]:
df_nvda = pd.read_csv('/Users/yuchen/Downloads/Nvidia-Tweets.csv')
df_nvda = df_nvda.dropna(subset=['Text'])  # Drop rows with missing 'Text' or 'Tweet Id'

import contractions

def preprocess_tweet(tweet):
    tweet = re.sub(r'http\S+|www\S+', '[URL]', tweet)  # Replace URLs with [URL]
    tweet = re.sub(r'#', '', tweet)  # Remove '#' from hashtags
    tweet = re.sub(r'([!?.])\1+', r'\1', tweet)  # Normalize repeated punctuation
    tweet = tweet.lower()  # Convert to lowercase
    tweet = re.sub(r'[^a-zA-Z\s]', '', tweet)  # Remove non-alphabetic characters
    tweet = re.sub(r'[^\x00-\x7F]+', '', tweet) #remove emojis
    tweet = contractions.fix(tweet)  # Expand contractions
    return tweet


df_nvda['Cleaned_Text'] = df_nvda['Text'].astype(str).apply(preprocess_tweet)


# Attempt to convert 'Datetime' column to datetime, invalid entries will become NaT (Not a Time)
df_nvda['Datetime'] = pd.to_datetime(df_nvda['Datetime'], errors='coerce')

# Drop rows where 'Datetime' could not be converted (NaT values)
df_nvda = df_nvda.dropna(subset=['Datetime'])

# Proceed with extracting the date
df_nvda['date'] = df_nvda['Datetime'].dt.date

# Check rows where Datetime is invalid (NaT)
invalid_rows = df_nvda[df_nvda['Datetime'].isna()]
print(invalid_rows)


# Get the oldest (earliest) date
oldest_date = df_nvda['date'].min()

# Get the most recent date
most_recent_date = df_nvda['date'].max()

print(f"Oldest Date: {oldest_date}")
print(f"Most Recent Date: {most_recent_date}")

df_nvda = df_nvda[~df_nvda['Datetime'].dt.weekday.isin([5, 6])]

# If you have a list of holidays, e.g., holidays in the format of datetime objects
holidays = pd.to_datetime(['2024-12-25', '2024-01-01'])  # Example of holidays
df_nvda = df_nvda[~df_nvda['date'].isin(holidays)]


Empty DataFrame
Columns: [Unnamed: 0, Datetime, Tweet Id, Text, Username, Cleaned_Text, date]
Index: []
Oldest Date: 2022-11-21
Most Recent Date: 2023-02-06


In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification

# Specify the path to your saved model and tokenizer files
model_path = "/Users/yuchen/Downloads/"

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained(model_path+'config.json')
model = BertForSequenceClassification.from_pretrained(model_path)

# If using GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare your dataset
input_data = df_nvda['Cleaned_Text'].tolist()
# Tokenize the entire dataset
inputs = tokenizer(input_data, padding=True, truncation=True, max_length=128, return_tensors="pt")

# Convert the tokenized data into a TensorDataset
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])

/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1925: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


In [152]:
# Define DataLoader to create batches
batch_size = 128
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [153]:
# List to store predictions for each batch
from tqdm import tqdm

# List to store predictions for each batch
predictions = []

# Process each batch with tqdm to track progress
for batch in tqdm(dataloader, desc="Processing Batches", unit="batch"):
    input_ids, attention_mask = batch
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    # Run the model
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    
    # Extract logits
    logits = outputs.logits
    
    # Convert logits to probabilities using softmax (for classification tasks)
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    
    # Get predicted class (for classification tasks)
    predicted_class = torch.argmax(probabilities, dim=-1)
    
    # Append predictions to the list
    predictions.extend(predicted_class.cpu().numpy())

# Save predictions to a DataFrame (or list of predicted classes)
df_predictions = pd.DataFrame(predictions, columns=['Predicted_Class'])

# Optionally, you can add the predictions to the original DataFrame (df_nvda)
df_nvda['Predicted_Class'] = df_predictions['Predicted_Class']

# Save to CSV or other file formats
df_nvda.to_csv('nvda_predictions.csv', index=False)

Processing Batches: 100%|████████████████| 650/650 [3:00:48<00:00, 16.69s/batch]


In [162]:
# Define the stock symbol and date range
symbol = 'NVDA'
start_date = '2022-11-21'
end_date = '2023-02-06'

# Get the stock data from Yahoo Finance
stock_data = yf.download(symbol, start=start_date, end=end_date)

# Select the 'Adj Close' column
adj_close = stock_data['Adj Close']

# Drop non-trading days (if any)
adj_close = adj_close.dropna()
from textblob import TextBlob

def get_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity  # returns a score between -1 and 1

df_nvda['Sentiment'] = df_nvda['Cleaned_Text'].apply(get_sentiment)

from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Apply sentiment analysis to the 'Cleaned_Text' column
df_nvda['Sentiment_Score'] = df_nvda['Cleaned_Text'].apply(lambda tweet: sia.polarity_scores(tweet)['compound'])
df_nvda['Predicted'] = df_nvda['Predicted_Class'].apply(lambda score: 1 if score == 1 else (-1 if score == 0 else 0))  
daily_data = df_nvda.groupby('date').agg({
    'Predicted':'sum',
    'Sentiment': 'sum',
    'Sentiment_Score':'sum',
    'Tweet Id': 'count'  # Count the number of tweets
}).reset_index()
# Classify sentiment

daily_data['SentimentScore_BLOB'] = daily_data['Sentiment']#.apply(lambda score: 1 if score > 0 else (-1 if score < 0 else 0))  
daily_data['SentimentScore_VADER'] = daily_data['Sentiment_Score']#.apply(lambda score: 1 if score > 0 else (-1 if score < 0 else 0))  
daily_data['SentimentScore_BERT'] = daily_data['Predicted']
daily_data.date = pd.to_datetime(daily_data.date)
adj_close.index = pd.to_datetime(adj_close.index) 
merged_df = pd.merge(daily_data, adj_close, left_on='date', right_index=True, how='left')

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
merged_df['price move'] = merged_df['Adj Close'].diff().shift(-1)
merged_df = merged_df.dropna()

X = merged_df[['Adj Close']]
y = merged_df['price move'].apply(lambda x: 1 if x > 0 else 0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

X = merged_df[['SentimentScore_BLOB','Adj Close']]
y = merged_df['price move'].apply(lambda x: 1 if x > 0 else 0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

X = merged_df[['SentimentScore_VADER','Adj Close']]
y = merged_df['price move'].apply(lambda x: 1 if x > 0 else 0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

X = merged_df[['SentimentScore_BERT','Adj Close']]
y = merged_df['price move'].apply(lambda x: 1 if x > 0 else 0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

[*********************100%%**********************]  1 of 1 completed

/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



              precision    recall  f1-score   support

           0       0.25      0.25      0.25         4
           1       0.50      0.50      0.50         6

    accuracy                           0.40        10
   macro avg       0.38      0.38      0.38        10
weighted avg       0.40      0.40      0.40        10

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.43      0.50      0.46         6

    accuracy                           0.30        10
   macro avg       0.21      0.25      0.23        10
weighted avg       0.26      0.30      0.28        10

              precision    recall  f1-score   support

           0       0.25      0.25      0.25         4
           1       0.50      0.50      0.50         6

    accuracy                           0.40        10
   macro avg       0.38      0.38      0.38        10
weighted avg       0.40      0.40      0.40        10

              prec

In [163]:
merged_df['daily_return'] = merged_df['Adj Close'].pct_change()

# Assume starting capital
initial_capital = 1000
merged_df['daily_profit'] = initial_capital * merged_df['daily_return']  # Profit for fixed capital
merged_df['cumulative_profit'] = merged_df['daily_profit'].cumsum()

# Metrics
total_profit = merged_df['daily_profit'].sum()
average_daily_return = merged_df['daily_return'].mean()
win_rate = (merged_df['daily_profit'] > 0).mean() * 100
max_drawdown = (merged_df['cumulative_profit'].cummax() - merged_df['cumulative_profit']).max()

# Print results
print(f"Total Profit: ${total_profit:.2f}")
print(f"Average Daily Return: {average_daily_return:.2%}")
print(f"Win Rate: {win_rate:.2f}%")
print(f"Maximum Drawdown: ${max_drawdown:.2f}")

Total Profit: $384.48
Average Daily Return: 0.85%
Win Rate: 56.52%
Maximum Drawdown: $244.86


In [142]:
"""
from torch.utils.data import DataLoader, TensorDataset
import torch
import pandas as pd

# Prepare your dataset
input_data = df_nvda['Cleaned_Text'].tolist()
# Tokenize the entire dataset
inputs = tokenizer(input_data, padding=True, truncation=True, max_length=128, return_tensors="pt")

# Convert the tokenized data into a TensorDataset
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])

# Define DataLoader to create batches
batch_size = 8
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# List to store predictions for each batch
predictions = []

# Process each batch
for batch in dataloader:
    input_ids, attention_mask = batch
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    
    # Run the model
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    
    # Extract logits
    logits = outputs.logits
    
    # Convert logits to probabilities using softmax (for classification tasks)
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    
    # Get predicted class (for classification tasks)
    predicted_class = torch.argmax(probabilities, dim=-1)
    
    # Append predictions to the list
    predictions.extend(predicted_class.cpu().numpy())

# Save predictions to a DataFrame (or list of predicted classes)
df_predictions = pd.DataFrame(predictions, columns=['Predicted_Class'])

# Optionally, you can add the predictions to the original DataFrame (df_nvda)
df_nvda['Predicted_Class'] = df_predictions['Predicted_Class']

# Save to CSV or other file formats
df_nvda.to_csv('nvda_predictions.csv', index=False)
"""


In [159]:
df_nvda['Predicted'].unique()

array([0])